In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
from datetime import datetime,timedelta
import pandas as pd
import numpy as np
import time
import requests


import warnings
warnings.filterwarnings("ignore")

def access_token(api_secret, request_token, kite):
    user = pd.DataFrame()
    try:
        user = kite.generate_session(request_token, api_secret)
       
        print("Access token ->" + user["access_token"])
        acc_key=user["access_token"]
   
    except Exception as e:
        print("Authentication failed", str(e))
        raise

    print(user["user_name"], "has successfully signed in.")
    return user



acc_key=0

while(acc_key==0):
    user_name = "RK2267"
    password = "Shan@#2021"
#     totp_key = "RGHL2AKPKBTQUPR35WFKWDTIO5NJEALQ"
    options = Options()
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disble-dev-shm-usage')
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get("https://kite.trade/connect/login?api_key=pv2830q1vbrhu1eu")
    time.sleep(1)
    element = driver.find_element_by_id("userid")
    element.send_keys(user_name)
    element = driver.find_element_by_id("password")
    element.send_keys(password)
    element.send_keys(Keys.RETURN)
    time.sleep(15)
#     outh = pyotp.TOTP(totp_key)
#     element = driver.find_element_by_id('externaltotp')
#     element.send_keys(outh.now())
#     element.send_keys(Keys.RETURN)
#     time.sleep(1)
    # element.close()
    print(driver.current_url)
    url = driver.current_url.split("request_token=")[1].split("&")[0]
    print(url, "new url")
    driver.quit()
    my_api_key = "pv2830q1vbrhu1eu"
    kite = KiteConnect(api_key=my_api_key)
    req_key = url
    user_details = access_token(
        "gz426yatawim9xruj2p6e51c40k44r7n", req_key, kite)
    acc_key = user_details["access_token"]
#     print("Access token ->" + acc_key)
    kite.set_access_token(acc_key)

https://kite.trade/?action=login&type=login&status=success&request_token=Dvj77wwshTPJ4Hmxr1bKnlR1vePgfEDj
Dvj77wwshTPJ4Hmxr1bKnlR1vePgfEDj new url
Access token ->Bnmmlp2PbTwgwU2CMkPGz8hj5aYb0xhW
Kakal Krishna Rao has successfully signed in.


In [12]:
from datetime import *

def history_data(ticker_val):
    data1=['' for i in range(1)]
    start_date=(datetime.now() - timedelta(days=100))
    curr_date=datetime.now()
    if 9<=curr_date.hour<16:
        new_date=(datetime.now() - timedelta(days=1))
    else:
        new_date=(datetime.now())
    
    startd=pd.to_datetime(str(start_date)[:10]+" "+'00:00:00.0')
    endd=pd.to_datetime(str(new_date)[:10]+' '+'15:30:00.0')
    
    url1="https://api.kite.trade/instruments/historical/"+str(ticker_val)+"/day?from="+str(startd)+"&to="+str(endd)
    #print(url1)
    HEADERS = {"X-Kite-Version": "3", "Authorization":"token pv2830q1vbrhu1eu:"+acc_key}
    res1 = requests.get(url1, headers=HEADERS)
    i=0
    data1[i] = res1.json()
    data1[i] = data1[i]["data"]["candles"]
    
    data1[i]=pd.DataFrame(data1[i])
    data1[i] = data1[i].rename(columns={0: 'Time', 1: 'Open', 2:'High', 3:'Low', 4:'Close', 5:'Volume'})
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    print('History_data {} collected .'.format(ticker_val))
    return data1

In [13]:
def EMA(df, base, target, period, alpha=False):
    con = pd.concat([df[:period][base].rolling(window=period).mean(), df[period:][base]])

    if (alpha == True):
        # (1 - alpha) * previous_val + alpha * current_val where alpha = 1 / period
        df[target] = con.ewm(alpha=1 / period, adjust=False).mean()
    else:
        # ((current_val - previous_val) * coeff) + previous_val where coeff = 2 / (period + 1)
        df[target] = con.ewm(span=period, adjust=False).mean()

    df[target].fillna(0, inplace=True)
    return df

def ATR(df, period, ohlc=['open', 'high', 'low', 'close']):
    atr = 'ATR_' + str(period)

    # Compute true range only if it is not computed and stored earlier in the df
    if not 'TR' in df.columns:
        df['h-l'] = df[ohlc[1]] - df[ohlc[2]]
        df['h-yc'] = abs(df[ohlc[1]] - df[ohlc[3]].shift())
        df['l-yc'] = abs(df[ohlc[2]] - df[ohlc[3]].shift())

        df['TR'] = df[['h-l', 'h-yc', 'l-yc']].max(axis=1)

        df.drop(['h-l', 'h-yc', 'l-yc'], inplace=True, axis=1)

    # Compute EMA of true range using ATR formula after ignoring first row
    EMA(df, 'TR', atr, period, alpha=True)

    return df

def SuperTrend(df, period, multiplier, ohlc=['Open', 'High', 'Low', 'Close']):

    ATR(df, period, ohlc=ohlc)
    atr = 'ATR_' + str(period)
    st = 'ST' #+ str(period) + '_' + str(multiplier)
    stx = 'STX' #  + str(period) + '_' + str(multiplier

    # Compute basic upper and lower bands
    df['basic_ub'] = (df[ohlc[1]] + df[ohlc[2]]) / 2 + multiplier * df[atr]
    df['basic_lb'] = (df[ohlc[1]] + df[ohlc[2]]) / 2 - multiplier * df[atr]

    # Compute final upper and lower bands
    df['final_ub'] = 0.00
    df['final_lb'] = 0.00
    for i in range(period, len(df)):
        df['final_ub'].iat[i] = df['basic_ub'].iat[i] if df['basic_ub'].iat[i] < df['final_ub'].iat[i - 1] or \
                                                         df[ohlc[3]].iat[i - 1] > df['final_ub'].iat[i - 1] else \
        df['final_ub'].iat[i - 1]
        df['final_lb'].iat[i] = df['basic_lb'].iat[i] if df['basic_lb'].iat[i] > df['final_lb'].iat[i - 1] or \
                                                         df[ohlc[3]].iat[i - 1] < df['final_lb'].iat[i - 1] else \
        df['final_lb'].iat[i - 1]

    # Set the Supertrend value
    df[st] = 0.00
    for i in range(period, len(df)):
        df[st].iat[i] = df['final_ub'].iat[i] if df[st].iat[i - 1] == df['final_ub'].iat[i - 1] and df[ohlc[3]].iat[
            i] <= df['final_ub'].iat[i] else \
            df['final_lb'].iat[i] if df[st].iat[i - 1] == df['final_ub'].iat[i - 1] and df[ohlc[3]].iat[i] > \
                                     df['final_ub'].iat[i] else \
                df['final_lb'].iat[i] if df[st].iat[i - 1] == df['final_lb'].iat[i - 1] and df[ohlc[3]].iat[i] >= \
                                         df['final_lb'].iat[i] else \
                    df['final_ub'].iat[i] if df[st].iat[i - 1] == df['final_lb'].iat[i - 1] and df[ohlc[3]].iat[i] < \
                                             df['final_lb'].iat[i] else 0.00

        # Mark the trend direction up/down
    df[stx] = np.where((df[st] > 0.00), np.where((df[ohlc[3]] < df[st]), 'down', 'up'), np.NaN)

    # Remove basic and final bands from the columns
    df.drop(['basic_ub', 'basic_lb', 'final_ub', 'final_lb'], inplace=True, axis=1)

    df.fillna(0, inplace=True)
    return 

In [14]:
from termcolor import colored
def crossover(dfx,tkname):
    SuperTrend(dfx,7,1)
    def change(s):
        return s[:10]

    dfx['Time']=dfx.Time.apply(change)
    last10=dfx.loc[len(dfx)-11:]
    buy=0
    sell=0
    nocross=0
    print('CALCULATING 10DAYS CROSSOVER OF',tkname)
    last10.reset_index(inplace=True,drop=True)
    print()
    for i in range(1,len(last10)):
        print('Date->',last10['Time'].iloc[i],'LTP->',last10['Close'].iloc[i],'ST ->',last10['ST'].iloc[i])
    print()
    for i in range(1,len(last10)):
        if last10['Close'].iloc[i]>last10['ST'].iloc[i] and last10['Close'].iloc[i-1]<last10['ST'].iloc[i-1]:
            print(colored('LONG ENTRY on {}'.format(last10['Time'].iloc[i]),'green'))
            buy+=1
        elif last10['Close'].iloc[i]<last10['ST'].iloc[i] and last10['Close'].iloc[i-1]>last10['ST'].iloc[i-1]:
            print(colored('SHORT ENTRY on {}'.format(last10['Time'].iloc[i]),'red'))
            sell+=1
        else:
            print(colored('No crossover on {}'.format(last10['Time'].iloc[i]),'blue'))
            nocross+=1

    print()
    print('--------- SUMMARY ---------------')
    print('Start date ->',last10['Time'].iloc[1])
    print('No of Long Entry :',buy)
    print('No of Short Entry :',sell)
    print('End date ->',last10['Time'].iloc[-1])
    print('----------------------------------')
    print()
    
    
    
dicct={67329: 'ATUL', 5436929: 'AUBANK', 70401: 'AUROPHARMA', 1510401: 'AXISBANK', 4267265: 'BAJAJ-AUTO', 4268801: 'BAJAJFINSV', 81153: 'BAJFINANCE', 85761: 'BALKRISIND', 87297: 'BALRAMCHIN', 579329: 'BANDHANBNK', 1195009: 'BANKBARODA', 94977: 'BATAINDIA', 98049: 'BEL', 103425: 'BERGEPAINT', 108033: 'BHARATFORG', 2714625: 'BHARTIARTL', 112129: 'BHEL', 2911489: 'BIOCON', 558337: 'BOSCHLTD', 134657: 'BPCL'}
tokens=[67329, 5436929, 70401, 1510401, 4267265, 4268801, 81153, 85761, 87297, 579329, 1195009, 94977, 98049, 103425, 108033, 2714625, 112129, 2911489, 558337, 134657]

for tk in tokens:
    df=history_data(tk)[0]
    crossover(df,dicct[tk])

History_data 67329 collected .
CALCULATING 10DAYS CROSSOVER OF ATUL

Date-> 2022-09-30 LTP-> 9023.9 ST -> 9063.9252012544
Date-> 2022-10-03 LTP-> 8919.6 ST -> 9063.9252012544
Date-> 2022-10-04 LTP-> 9039.55 ST -> 9063.9252012544
Date-> 2022-10-06 LTP-> 9001.7 ST -> 9063.9252012544
Date-> 2022-10-07 LTP-> 8900.4 ST -> 9063.9252012544
Date-> 2022-10-10 LTP-> 8767.95 ST -> 9032.22574260039
Date-> 2022-10-11 LTP-> 8671.85 ST -> 8911.457779371764
Date-> 2022-10-12 LTP-> 8696.5 ST -> 8870.078096604368
Date-> 2022-10-13 LTP-> 8763.2 ST -> 8870.078096604368
Date-> 2022-10-14 LTP-> 8764.75 ST -> 8870.078096604368

No crossover on 2022-09-30
No crossover on 2022-10-03
No crossover on 2022-10-04
No crossover on 2022-10-06
No crossover on 2022-10-07
No crossover on 2022-10-10
No crossover on 2022-10-11
No crossover on 2022-10-12
No crossover on 2022-10-13
No crossover on 2022-10-14

--------- SUMMARY ---------------
Start date -> 2022-09-30
No of Long Entry : 0
No of Short Entry : 0
End date -> 20

History_data 87297 collected .
CALCULATING 10DAYS CROSSOVER OF BALRAMCHIN

Date-> 2022-09-30 LTP-> 347.7 ST -> 355.5888374380609
Date-> 2022-10-03 LTP-> 344.7 ST -> 355.5888374380609
Date-> 2022-10-04 LTP-> 348.7 ST -> 355.5888374380609
Date-> 2022-10-06 LTP-> 360.4 ST -> 345.3194345084941
Date-> 2022-10-07 LTP-> 362.95 ST -> 350.88094386442356
Date-> 2022-10-10 LTP-> 357.9 ST -> 350.88094386442356
Date-> 2022-10-11 LTP-> 351.6 ST -> 350.88094386442356
Date-> 2022-10-12 LTP-> 358.85 ST -> 350.88094386442356
Date-> 2022-10-13 LTP-> 353.9 ST -> 350.88094386442356
Date-> 2022-10-14 LTP-> 349.5 ST -> 365.8908404539919

No crossover on 2022-09-30
No crossover on 2022-10-03
No crossover on 2022-10-04
LONG ENTRY on 2022-10-06
No crossover on 2022-10-07
No crossover on 2022-10-10
No crossover on 2022-10-11
No crossover on 2022-10-12
No crossover on 2022-10-13
SHORT ENTRY on 2022-10-14

--------- SUMMARY ---------------
Start date -> 2022-09-30
No of Long Entry : 1
No of Short Entry : 1
End dat

History_data 112129 collected .
CALCULATING 10DAYS CROSSOVER OF BHEL

Date-> 2022-09-30 LTP-> 59.9 ST -> 57.19245363916184
Date-> 2022-10-03 LTP-> 60.55 ST -> 58.81496026213872
Date-> 2022-10-04 LTP-> 62.0 ST -> 59.74139451040462
Date-> 2022-10-06 LTP-> 63.85 ST -> 61.29619529463253
Date-> 2022-10-07 LTP-> 63.9 ST -> 61.528881681113596
Date-> 2022-10-10 LTP-> 63.2 ST -> 61.528881681113596
Date-> 2022-10-11 LTP-> 60.95 ST -> 64.23031141795735
Date-> 2022-10-12 LTP-> 61.6 ST -> 63.00098121539202
Date-> 2022-10-13 LTP-> 60.65 ST -> 63.00098121539202
Date-> 2022-10-14 LTP-> 61.55 ST -> 63.00098121539202

No crossover on 2022-09-30
No crossover on 2022-10-03
No crossover on 2022-10-04
No crossover on 2022-10-06
No crossover on 2022-10-07
No crossover on 2022-10-10
SHORT ENTRY on 2022-10-11
No crossover on 2022-10-12
No crossover on 2022-10-13
No crossover on 2022-10-14

--------- SUMMARY ---------------
Start date -> 2022-09-30
No of Long Entry : 0
No of Short Entry : 1
End date -> 2022-10-